In [18]:
import os
import torch
import torchaudio
import numpy as np
import math

In [19]:
folder="Convolution"
subfolder="mic_signal"

file_path=os.path.join(folder,subfolder)

# wav file 불러오기
file_list=os.listdir(file_path)
file_list

['mic1.wav', 'mic6.wav', 'mic5.wav', 'mic4.wav', 'mic2.wav', 'mic3.wav']

In [13]:
src_angle=60
speed_of_sound = 343  
distance = 0.07

In [15]:
beamformed=torch.empty(1, 1025, 519)
for i in range(len(file_list)):
    waveform, sample_rate = torchaudio.load(os.path.join(file_path,file_list[i]))
    stft_result = torch.stft(waveform, n_fft=2048, hop_length=512,return_complex=True)
    real_part,imag_part=stft_result.real,stft_result.imag
    magnitude,phase=torch.sqrt(real_part**2 + imag_part**2),torch.atan2(imag_part, real_part)
    magnitude,phase=magnitude[0], phase[0]

    freq_index=stft_result[0].shape[0]
    time_index=stft_result[0].shape[1]
    phase_delay=torch.zeros(freq_index, time_index)
    complex_signal=torch.zeros(freq_index, time_index)

    time_delay=distance*math.cos(math.radians(src_angle))/speed_of_sound
    total_freqbin=stft_result[0]

    for freq in range(freq_index):
        for time in range(time_index):
            # freqbin마다 phase delay 계산
            phase_delay[freq][time]=total_freqbin[freq][time]*2*math.pi*time_delay
            # phase와 phase delay 차이 계산하여 복소수 변환
            complex_signal[freq][time]=magnitude[freq][time]*torch.exp(1j*(phase[freq][time]-phase_delay[freq][time]))

    complex_signal_dim = complex_signal.unsqueeze(0)
    istft_result = torch.istft(complex_signal_dim,n_fft=2048,hop_length=512,return_complex=True)
    beamformed+=istft_result

RuntimeError: istft requires a complex-valued input tensor matching the output from stft with return_complex=True.

In [17]:
complex_signal.shape

torch.Size([1025, 360])

In [149]:
torchaudio.save("Beamformed","beamformed_signal.wav", beamformed/len(file_list), sample_rate)

tensor([[-4.3038e-03, -1.0904e-02,  2.7122e-02,  ...,  6.4925e-03,
          2.6259e-01,  2.9510e-01],
        [ 5.4271e-02,  1.1388e-03, -2.6133e-02,  ...,  1.3351e-02,
          2.4293e-01, -2.7608e-02],
        [ 8.2528e-02, -6.5621e-02,  4.1663e-02,  ...,  2.7898e-02,
          1.4318e-01,  6.8262e-03],
        ...,
        [ 9.5807e-04, -2.8514e-04, -5.1851e-04,  ...,  5.1615e-04,
         -8.0829e-04, -1.8932e-03],
        [ 3.7774e-04,  6.1100e-04, -1.0115e-03,  ...,  1.5755e-04,
          2.6796e-05, -2.4420e-03],
        [-1.8442e-04,  2.9439e-04,  4.9863e-06,  ...,  1.7202e-03,
         -4.0323e-05, -2.9063e-03]])